In [9]:
# Import libraries
import logging
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import logging
from itertools import chain

In [43]:
# helper method for extracting data of the player

def extract_info(tr):
    base = "https://sofifa.com/"
    return {
        "link": base + tr.select('td.col-name')[0].find("a").get("href"),
        "name": tr.select('td.col-name')[0].find("a").get("aria-label"),
        "country": tr.select('td.col-name')[0].find("img").get("title"),
        "age": tr.select('td.col.col-ae')[0].text.strip(),
        "overall": tr.select('td.col.col-oa')[0].text.strip(),
        "potential": tr.select('td.col.col-pt')[0].text.strip(),
        "club": tr.select("td.col-name")[1].find("a").text,
        "best_position": tr.select('td.col-name')[0].find("span").text,
        "value": tr.select('td.col.col-vl')[0].text.strip(),
        "wage": tr.select('td.col.col-wg')[0].text.strip(),
    }

# helper method for extracting stats of single player
def extract_stats(tr):
    extract_att() # implement
    extract_skill() # implement
    extract_move() # implement
    extract_pow() # implement
    extract_mentality() # implement
    extract_def() # implement
    extract_goalkeep() # implement

In [44]:
import threading


class MultiThreading(object):

    def __init__(self, scrapers):
        self.scrapers = scrapers

    def run(self):
        threads = []

        for i in range(len(self.scrapers)):
            t = threading.Thread(target=self.scrapers[i].start)
            t.start()
            threads.append(t)

        for thread in threads:
            thread.join()

In [45]:
class Scraper(object):
    """
    Pull player info down from web
    """

    # Initialize array to store player (surface)
    players_scraped = []

    # Initialize array to store player stats (deep)
    stats_scraped = []

    # Instantiate scraper
    def __init__(self, urls):
        self.urls = urls
        self.logger = logging.getLogger("sLogger")

    # request to get the url
    def get_page(self, url):
        response = requests.get(url)
        if response.status_code:
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.find("tbody", {"class": "list"})
        else:
            self.logger.error("Error" + response.status_code)
            return None

    # helper method to get players
    def get_players(self,trs):
        return [extract_info(tr) for tr in trs]
    
    # helper method to get stats
    def get_stats(self,trs):
        return [extract_stats(tr) for tr in trs]

    # method to extract and copy player info from web
    def scrap(self, urls):
        for url in urls:
            tbody = self.get_page(url)
            if tbody is None:
                continue
            trs = tbody.findAll("tr")
            Scraper.players_scraped.append(self.get_players(trs))
            self.logger.info("Page{} scraped".format(len(Scraper.players_scraped)))
    # method to start the scraper
    def start(self):
        self.scrap(self.urls)

In [46]:
params = {"ae": "0", "oa": "1", "pt": "2", "vl": "3", "wg": "4", "bp": "5"}
query = "&".join([f"showCol%5B{y}%5D={x}" for x, y in params.items()])
url = f"https://sofifa.com/players?{query}&offset="
urls = [url + str(offset) for offset in range(0, 120, 60)]
# Parameters
number_of_scraper = 31
pages = 10

scrapers = [Scraper(urls[pages * i:min(pages * (i + 1), len(urls))]) 
            for i in range(number_of_scraper)]
multi_threading = MultiThreading(scrapers)

#     multi_threading.run()


In [ ]:
scrap2 = [Scraper(urls[pages * i:min(pages * (i + 1), len(p_urls))]) 
            for i in range(number_of_scraper)]
multi_2 = MultiThreading(scrap2)
multi_2.run()

In [73]:
t1 = time.time()
multi_threading.run()
flatten = lambda x: list(chain.from_iterable(x))
df_multi_thread = pd.DataFrame(flatten(Scraper.players_scraped))
df_multi_thread.drop_duplicates(inplace=True, ignore_index=True)
p_urls = df_multi_thread[["link"]].values.tolist()
df_multi_thread = df_multi_thread.drop("link",axis = 1)

print("Multi threading time taken: ", time.time() - t1)
df_multi_thread.head()


Multi threading time taken:  1.0103487968444824


,name,country,age,overall,potential,club,best_position,value,wage
0,Ludovit Reis,Netherlands,21,71,79,Hamburger SV,CM,€4.2M,€7K
1,Dele Alli,England,25,78,81,Everton,CAM,€18.5M,€72K
2,Tokmac Chol Nguen,Norway,27,74,74,Ferencvárosi TC,CAM,€4.5M,€1K
3,Salih Özcan,Turkey,23,77,81,1. FC Köln,CDM,€14M,€23K
4,Lukas Podolski,Germany,36,72,72,Górnik Zabrze,CF,€800K,€6K


In [128]:
test_url = p_urls[0][0]
test_url

stats = []
res = requests.get(test_url)
text = res.text
soup = BeautifulSoup(text, "html.parser")
tbody = soup.find_all("div", {"class":"center"})[5]

stats_block = tbody.findAll("div", {"class":"block-quarter"})
att_block = stats_block[0]
skill_block = stats_block[1]
movement_block = stats_block[2]
power_block = stats_block[3]
mentality_block = stats_block[4]
defending_block = stats_block[5]
goalkeeping_block = stats_block[6]
#stats.append([extract_stats(tr) for tr in trs])

# Convert list of lists to single list
#flatten = lambda x: list(chain.from_iterable(x))

# Convert to df
# df = pd.DataFrame(flatten(stats))
# df.drop_duplicates(inplace=True, ignore_index=True)

# df.head()

## Prototype

- Refactor the Scraper class, so that it has different branch of statements to handle either scrap surface only or scrap deep only
- Implement extarct_stats (so that it calls 6 helpers to extract info according to attributes)